## Add log probs, check for corner case when its a start word

In [2]:
chapter = ""
with open ('Austen_Pride.txt','r') as f:
    for line in f:
        chapter += line
chapter = chapter.replace('\n', ' ').replace("ï»¿", "").strip("'").strip("`")

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from math import log10

In [4]:
chapter = 'truth universally hated. It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. "My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?" Mr. Bennet replied that he had not. "But it is," returned she; "for Mrs. Long has just been here, and she told me all about it."It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. "My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?" Mr. Bennet replied that he had not. "But it is," returned she; "for Mrs. Long has just been here, and she told me all about it."'
#chapter = "I love going to the lake. I love going to the lake. I went to the lake. I hate going to the lake. go to the mall. come to the mall"

In [28]:
class kneyserNey():
    '''
    Given an n
    '''
    
    '''import pandas as pd
    import numpy as np
    import nltk
    from math import log10
    '''
    def __init__(self):
        #setting in fit
        self.ngram_order = None
        self.all_gram = None
        self.vocab = None
        
    def make_ngrams(self, text, n):
        '''
        takes a text and n gram and creates an n-gram for it
        '''
        # Parse text into sentences
        sent_text = sent_tokenize(text)
        # Get n-grams
        dict_ngram = {}
        for sentence in sent_text:
            sentence = (n-1)*"<s> " + sentence # create n-1 pseudo tokens
            n_grams = nltk.ngrams(sentence.split(), n)
            for grams in n_grams:
                new_gram = []
                # Change infrequent words into unknown
                for word in grams:
                    word = word.strip(".")
                    wLower = word.lower()
                    new_gram.append(wLower)
                new_gram = tuple(new_gram)
                if new_gram in dict_ngram:
                    dict_ngram[new_gram] = dict_ngram[new_gram] + 1 
                else:
                    dict_ngram[new_gram] = 1
        return dict_ngram

    def fit(self, text, ngram_order):
        all_gram = {}
        for gram in range(1, ngram_order+1):
            all_gram[gram] = self.make_ngrams(text, gram)
        vocab = len(all_gram[1]) -1 # -1 to take care of start phrase
        self.ngram_order = ngram_order
        self.all_gram = all_gram
        self.vocab = vocab
        return self
    
    
    def score(self, phrase, d):
        ngram_len = len(phrase)
        all_gram = self.all_gram
        ngram_order = self.ngram_order
        vocab = self.vocab
        if ngram_len == 1: # base case
            if phrase in all_gram[ngram_len]:
                probability = all_gram[ngram_len][phrase]/vocab
                return probability
            else:
                return 1/(vocab + 1) # the word does not exist
        else: #recursive case    
            if ngram_len == ngram_order: ##counting  case
                if phrase in all_gram[ngram_len]:
                    num_1 = max(all_gram[ngram_len][phrase] - d, 0)
                    num_2 = len([each for each in all_gram[ngram_len] if phrase[:-1] == each[:-1]])
                    denom = sum([all_gram[ngram_len][each] for each in all_gram[ngram_len] if phrase[:-1] == each[:-1]])
                    probability = num_1/denom + d*num_2/denom*self.score(phrase[1:], d)
                    return probability
                else:
                    print(ngram_len, "counting + notfound")
                    probability = self.score(phrase[1:], d) # we check for one lower gram
                    return probability

            else: #continuous counting case
                if phrase in all_gram[ngram_len]:
                    num_1 = max(len([each for each in all_gram[ngram_len+1] if phrase == each[1:]]) - d, 0)
                    num_2 = len([each for each in all_gram[ngram_len] if phrase[:-1] == each[:-1]])
                    denom = len([each for each in all_gram[ngram_len + 1 ] if phrase[:-1] == each[1:-1]])
                    probability = num_1/denom + d*num_2/denom*self.score(phrase[1:], d)
                    return probability
                else:
                    print(ngram_len,"continn counting +not found")
                    probability = self.score(phrase[1:], d) # we check for one lower gram
                    return probability

In [21]:
phrase=('truth', 'universally', 'hated,')
ngram_order = 3
d = 0.75

In [25]:
prideKN = kneyserNey()

In [29]:
prideKN.fit(chapter, ngram_order)

In [30]:
prideKN.score(phrase, d)

3 counting + notfound
2 continn counting +not found
base + notfound


-1.9542425094393248

In [ ]:
text = "i am sam x. sam am i x. i am sam x. i do not like green eggs and sam x."

In [15]:
gram =prideKN.fit(text, 2).all_gram

NameError: name 'text' is not defined

In [14]:
len(gram[2])

NameError: name 'gram' is not defined